# Know Thyself: Using Data Science to Explore Your Own Genome
## DNA analysis with pandas and Selenium

[23andme](https://www.23andme.com/) once offered me a free DNA and ancestry test kit if I participated in one of their clinical studies. In exchange for my genetic data and a bunch of questionnaires, I got my genome sequenced and gained access to myriad reports on where my ancestors were likely from, what health conditions and traits I probably have inherited, and who else on the site I might be related to.

While 23andme provides an overwhelming amount of consumer-ready infographics and tools, I wondered what else I could do with the data. I knew I could download my raw genetic data from the site in a text file, so I decided to pour it into pandas and see what I could make of it. 

In [ ]:
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
sns.color_palette('Spectral')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import requests
import pandas as pd

import re

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait

## Importing my DNA into pandas and exploring the genome

Looking at the .txt file, I could see that I was missing some genotype values, which were denoted with '--'. 

While most of the chromosomes are ints, there is an X and a Y (which I may convert later to a number for analytic purposes), but for now I need to make sure to specify the data type properly so that pandas doesn't throw an error when it sees mixed data in the input. 

The other columns are fairly straightforward. I also want pandas to ignore the prefatory comments at the beginning of the file that consist of lines beginning with an octothorpe.

The arguments I need to pass are:
* separator (tab-delimited)
* dtype (as a dict)
* na_values ('--')
* comment ('#')

![img](/genome.png)

In [ ]:
data = pd.read_csv('genome.txt', sep='\t', dtype={'rsid':'str', 'chromosome':'object', 'position':'int', 'genotype':'str'}, comment='#')



In [ ]:
print(data.head())

A quick note on the column names:

* rsid stands for Reference SNP cluster ID. It identifies unique SNPs.

* SNPs are Single Nucleotide Polymorphisms ('snips'), locations in the genome that are known to vary between individuals. They can influence disease risk, drug efficacy and side-effects, tell you about your ancestry, and predict aspects of how you look and act.

* All humans have almost the same sequence of 3 billion DNA bases (A,C,G, or T) distributed between their 23 pairs of chromosomes. But at certain locations there are differences that have been reported to be meaningful, either medically or for other reasons (such as for genealogy). The SNPedia catalogues SNPs that have significant medical consequences, are common, are reproducible (or found in meta-analyses or studies of at least 500 patients), or have other historic or medical significance.

I started off by navigating my new data frame with some basic exploratory data analysis and data cleaning.

In [ ]:
# Read the data into a pandas DataFrame and do some EDA
df = pd.DataFrame(data)

In [ ]:
df.head(25)

In [ ]:
df.isna().any()

In [ ]:
df.nunique()

In [ ]:
df.info()

In [ ]:
duplicates = df[df.duplicated(subset='position')]
display(duplicates.head())
display(duplicates.info())

In [ ]:
# How many chromosomes am I missing by not having a Y chromosome?
Y_chromosome = df[df.chromosome == 'Y']

In [ ]:
len(Y_chromosome)

Since I don't have this chromosome, I'll just drop it from the DataFrame.

In [ ]:
# df = df[df.chromosome != 'Y']
df.info()

Most of the chromosomes are numeric; only X, Y, and mitochondrial are characters. I'll convert them to numbers, cast them to ints, and create a dictionary to translate back later so that the data will be more manipulable.

In [ ]:
df['chromosome'].unique()

In [ ]:
df['chromosome'] = df['chromosome'].apply(lambda x: re.sub(r'X', r'23', x))
df['chromosome'] = df['chromosome'].apply(lambda x: re.sub(r'Y', r'24', x))
df['chromosome'] = df['chromosome'].apply(lambda x: re.sub(r'MT', r'25', x))

In [ ]:
df['chromosome'] = df['chromosome'].apply(lambda x: int(x))

In [ ]:
chromosome_dict = {1:'1', 2:'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8', 9:'9', 10:'10', 11:'11', 12:'12', 13:'13', 
                  14:'14', 15:'15', 16:'16', 17:'17', 18:'18', 19:'19', 20:'20', 21:'21', 22:'22', 23:'X', 24:'Y', 25:'MT'}

In [ ]:
print(chromosome_dict)
df.info()

There are 16,005 genotypes that I simply do not have:

In [ ]:
genotype_na = df[df.genotype == '--']
len(genotype_na)

### Some visualizations

In [ ]:
df[df.chromosome == 1].info()

In [ ]:
# Remove that pesky whitespace from the column name
df.rename({' rsid': 'rsid'}, axis='columns', inplace=True)

How many SNPs are there per chromosome?

In [ ]:
# We can do this manually with a for loop . . .
x = []
y = []
for k in chromosome_dict:
    x.append(k)
    y.append(len(df[df.chromosome == k]))
rsid_per_chromosome = dict(zip(x,y)) 

In [ ]:
rsid_per_chromosome

In [ ]:
# . . . but pandas makes it a lot easier!
rsid_per_chromosome_series = df.groupby('chromosome')['rsid'].count()
rsid_per_chromosome_series.columns = ['chromosome', 'count']

In [ ]:
rsid_per_chromosome_series.plot.barh(figsize=(16,9), fontsize=15)
plt.show()

## Getting data on SNPs from SNPedia

To get some more info about my DNA, I pulled some information on clinically significant SNPs from SNPedia.

The columns are:

* Unnamed: 0 (actually the SNP name)
* Magnitude (a subjective measure of interest)
* Repute (a subjective measure of whether the genotype is "good" or "bad" to have based on research, and blank for things like ancestry and eye color)
* Summary (a narrative description)

In [ ]:
snp_df = pd.read_csv('results1.csv')
snp_df.head()

To match up with my original DataFrame, I'll create a genotype column and use regex to separate out the genotype, which is tacked onto the end of the SNP.

In [ ]:
snp_df['genotype'] = snp_df['Unnamed: 0'].apply(lambda x: re.sub(r'.*([AGCTID]);([AGCTID])\)', r'\1\2', x))

In [261]:
snp_df.head()

,rsid,magnitude,repute,summary,genotype
0,rs1801253,0.0,Good,NaN,GG
1,rs1801253,1.1,NaN,responds well to bucindolol; may also depend on rs1801252,CC
2,rs17822931,2.0,NaN,Wet earwax. Normal body odour. Normal colostrum.,CC
3,rs17822931,2.5,Good,Dry earwax. No body odour. Likely Asian ancestry. Reduced colostrum.,TT
4,rs16891982,1.1,Good,"generally non-European, but if European, 7x more likely to have black hair",CC


For consistency's sake, I renamed the columns to match my original DataFrame and made sure the rsids were all lower-case.

In [ ]:
new_cols = ['rsid', 'magnitude', 'repute', 'summary', 'genotype']
snp_df.columns = new_cols

I'll use regex to clean up the rsid a little more, too.

In [ ]:
snp_df['rsid'] = snp_df['rsid'].map(lambda x : x.lower())
snp_df['rsid'] = snp_df['rsid'].map(lambda x : re.sub(r'([a-z]{1,}[\d]+)\([agctid];[agctid]\)', r'\1', x))

In [262]:
snp_df.head()

,rsid,magnitude,repute,summary,genotype
0,rs1801253,0.0,Good,NaN,GG
1,rs1801253,1.1,NaN,responds well to bucindolol; may also depend on rs1801252,CC
2,rs17822931,2.0,NaN,Wet earwax. Normal body odour. Normal colostrum.,CC
3,rs17822931,2.5,Good,Dry earwax. No body odour. Likely Asian ancestry. Reduced colostrum.,TT
4,rs16891982,1.1,Good,"generally non-European, but if European, 7x more likely to have black hair",CC


In [263]:
snp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   rsid       10000 non-null  object 
 1   magnitude  10000 non-null  float64
 2   repute     8400 non-null   object 
 3   summary    8884 non-null   object 
 4   genotype   10000 non-null  object 
dtypes: float64(1), object(4)
memory usage: 390.8+ KB


Let's see what's going on with the null reputes and summaries and overwrite them if it's appropriate. (In this case, the answer seems to be yes.)

In [264]:
null_repute = snp_df[snp_df['repute'].isnull()]
null_summaries = snp_df[snp_df['summary'].isnull()]
null_repute_and_summaries = pd.concat([null_repute,null_summaries]).drop_duplicates().reset_index(drop=True)
display(null_repute_and_summaries)

,rsid,magnitude,repute,summary,genotype
0,rs1801253,1.1,NaN,responds well to bucindolol; may also depend on rs1801252,CC
1,rs17822931,2.0,NaN,Wet earwax. Normal body odour. Normal colostrum.,CC
2,rs16891982,1.1,NaN,Generally European; Light skin; Possibly an increased risk of melanoma,GG
3,rs1426654,2.7,NaN,"probably light-skinned, European ancestry",AA
4,rs1426654,2.6,NaN,"probably darker-skinned, Asian or African ancestry",GG
...,...,...,...,...,...
1694,rs11964281,0.0,Good,NaN,CC
1695,rs3774923,0.0,Good,NaN,GG
1696,rs28371694,0.0,Good,NaN,CC
1697,rs2619522,0.0,Good,NaN,TT


In [265]:
snp_df['repute'].fillna(value='Neutral', inplace=True)
snp_df['summary'].fillna(value='None', inplace=True)

In [266]:
snp_df[snp_df.rsid.str.contains('3003626')]

,rsid,magnitude,repute,summary,genotype
2271,i3003626,0.0,Bad,None,II
2273,i3003626,5.0,Good,resistant to HIV,DD
2284,i3003626,2.0,Good,None,DI


In [267]:
# No no NaNette
snp_df.isna().any()

rsid         False
magnitude    False
repute       False
summary      False
genotype     False
dtype: bool

In [260]:
df[df.rsid=='i3003626']

,rsid,chromosome,position,genotype
112130,i3003626,3,46414947,DI


# Merging my data with SNPedia

Here, I've done an inner join of the SNPedia DataFrame on my DNA to see what data, if any, it has on my particular genotypes.

In [268]:
new_df = snp_df.merge(df, how='inner', on=['rsid', 'genotype'], suffixes=('_SNPedia', '_myDNA'))

In [269]:
new_df.head(20)

,rsid,magnitude,repute,summary,genotype,chromosome,position
0,rs1801253,1.1,Neutral,responds well to bucindolol; may also depend on rs1801252,CC,10,115805056
1,rs17822931,2.0,Neutral,Wet earwax. Normal body odour. Normal colostrum.,CC,16,48258198
2,rs16891982,1.1,Neutral,Generally European; Light skin; Possibly an increased risk of melanoma,GG,5,33951693
3,rs1426654,2.7,Neutral,"probably light-skinned, European ancestry",AA,15,48426484
4,rs713598,1.1,Good,Can taste bitter.,CC,7,141673345
5,rs2227564,0.0,Good,common in clinvar,TT,10,75673101
6,rs4680,2.5,Good,"(warrior) multiple associations, see details",GG,22,19951271
7,rs3135506,0.0,Good,common on affy axiom data,GG,11,116662407
8,rs696217,0.0,Good,common in clinvar,GG,3,10331457
9,rs7412,1.0,Neutral,part of APOE4 genoset evaluation,CC,19,45412079


### What's hiding in there?

In [270]:
# Create a DataFrame for some subsets of genes
good_genes = new_df[(new_df.repute == 'Good') & (new_df.magnitude != 0)]
bad_genes = new_df[(new_df.repute == 'Bad') & (new_df.magnitude !=0)]
interesting_genes = new_df[new_df.magnitude > 4] # 4 is the threshold for "worth your time" given by SNPedia

In [271]:
useless = '|'.join(['common', 'common in clinvar', 'common in complete genomics', 'average', 'Normal', 'normal', 'normal risk', 'None']).lower()

In [272]:
#uncommon = new_df[~new_df.summary.isin(['common', 'common in clinvar', 'common in complete genomics', 'average', 'Normal', 'normal', 'normal risk', 'None'])]
uncommon = new_df[
    ~(new_df.summary.str.lower().str.contains(useless, flags=re.IGNORECASE, regex=True))]
      
      

In [275]:
new_df[new_df.rsid.str.contains('3003626')]

,rsid,magnitude,repute,summary,genotype,chromosome,position
381,i3003626,2.0,Good,None,DI,3,46414947


In [281]:
new_df[(new_df.repute == 'Good') & (new_df.magnitude >= 1)]

,rsid,magnitude,repute,summary,genotype,chromosome,position
4,rs713598,1.1,Good,Can taste bitter.,CC,7,141673345
6,rs4680,2.5,Good,"(warrior) multiple associations, see details",GG,22,19951271
13,rs1061147,1.0,Good,Normal 0.97x risk for Age Related Macular Degeneration.,AC,1,196654324
23,rs5882,2.1,Good,"Longer lifespan, 0.28x lower risk of dementia, 0.31x lower risk of Alzheimer's.",GG,16,57016092
112,rs671,2.0,Good,"Alcohol Flush: Normal, doesn't flush. Normal hangovers. Normal risk of Alcoholism. Normal risk ...",GG,12,112241766
133,rs9264942,2.1,Good,60% reduction in HIV viral load,CT,6,31274380
135,rs1799853,1.0,Good,normal; no risk of altered warfarin metabolism or NSAID metabolism,CC,10,96702047
154,rs601338,2.5,Good,resistance to Norovirus infection,AA,19,49206674
167,rs6601764,1.0,Good,Normal risk of developing Crohn's disease,TT,10,3862542
183,rs17070145,2.0,Good,increased memory performance,CT,5,167845791


I have plenty of "good" genotypes, but none with a nonzero magnitude.

In [ ]:
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 99

In [276]:
uncommon

,rsid,magnitude,repute,summary,genotype,chromosome,position
0,rs1801253,1.10,Neutral,responds well to bucindolol; may also depend on rs1801252,CC,10,115805056
2,rs16891982,1.10,Neutral,Generally European; Light skin; Possibly an increased risk of melanoma,GG,5,33951693
3,rs1426654,2.70,Neutral,"probably light-skinned, European ancestry",AA,15,48426484
4,rs713598,1.10,Good,Can taste bitter.,CC,7,141673345
6,rs4680,2.50,Good,"(warrior) multiple associations, see details",GG,22,19951271
9,rs7412,1.00,Neutral,part of APOE4 genoset evaluation,CC,19,45412079
14,rs1061170,2.50,Bad,2.5x risk for AMD; higher mortality among nonagenarians,CT,1,196659237
18,rs17580,2.50,Bad,a slightly reduced functionality form of Alpha-1 Antitrypsin Deficiency,TT,14,94847262
23,rs5882,2.10,Good,"Longer lifespan, 0.28x lower risk of dementia, 0.31x lower risk of Alzheimer's.",GG,16,57016092
24,rs2230199,2.00,Bad,2.5x+ risk of ARMD,GG,19,6718387


In [277]:
good_genes

,rsid,magnitude,repute,summary,genotype,chromosome,position
4,rs713598,1.1,Good,Can taste bitter.,CC,7,141673345
6,rs4680,2.5,Good,"(warrior) multiple associations, see details",GG,22,19951271
13,rs1061147,1.0,Good,Normal 0.97x risk for Age Related Macular Degeneration.,AC,1,196654324
23,rs5882,2.1,Good,"Longer lifespan, 0.28x lower risk of dementia, 0.31x lower risk of Alzheimer's.",GG,16,57016092
112,rs671,2.0,Good,"Alcohol Flush: Normal, doesn't flush. Normal hangovers. Normal risk of Alcoholism. Normal risk ...",GG,12,112241766
133,rs9264942,2.1,Good,60% reduction in HIV viral load,CT,6,31274380
134,rs1057910,0.1,Good,normal; no effect on warfarin metabolism,AA,10,96741053
135,rs1799853,1.0,Good,normal; no risk of altered warfarin metabolism or NSAID metabolism,CC,10,96702047
136,rs1801282,0.1,Good,normal fat metabolism; common in clinvar,CC,3,12393125
154,rs601338,2.5,Good,resistance to Norovirus infection,AA,19,49206674


I have three "bad" genotypes with a nonzero magnitude.

In [278]:
bad_genes

,rsid,magnitude,repute,summary,genotype,chromosome,position
14,rs1061170,2.50,Bad,2.5x risk for AMD; higher mortality among nonagenarians,CT,1,196659237
18,rs17580,2.50,Bad,a slightly reduced functionality form of Alpha-1 Antitrypsin Deficiency,TT,14,94847262
24,rs2230199,2.00,Bad,2.5x+ risk of ARMD,GG,19,6718387
47,rs28936383,5.00,Bad,Limb-girdle muscular dystrophy-dystroglycanopathy,GG,4,52895065
99,rs3743930,2.10,Bad,some reports of familial Mediterranean fever,CC,16,3304626
115,rs28930069,5.00,Bad,Hypokalemic periodic paralysis risk,GG,1,201022667
129,rs9282671,1.50,Bad,small chance of slightly increasing risk for a form of glaucoma,AA,2,38302291
141,rs1333049,3.00,Bad,1.5x increased risk for CAD,CG,9,22125503
142,rs8055236,1.70,Bad,"common, but 2.2x higher risk for heart disease",GG,16,83212398
146,rs1800462,3.50,Bad,incapable of detoxifying certain drugs,CC,6,18143955


Sadly I have no "interesting" genotypes above the threshold of 4, although I have some slightly interesting bad ones.

In [279]:
interesting_genes

,rsid,magnitude,repute,summary,genotype,chromosome,position
47,rs28936383,5.0,Bad,Limb-girdle muscular dystrophy-dystroglycanopathy,GG,4,52895065
115,rs28930069,5.0,Bad,Hypokalemic periodic paralysis risk,GG,1,201022667
715,rs62514958,5.9,Bad,Non-phenylketonuria hyperphenylalaninemia genotype,GG,12,103240677


# Scrape relevant articles with Selenium

Now I'd like to read up on my bad genetics, so I'll use Selenium to grab the abstracts of some scientific papers for me from PubMed.

In [ ]:
# Get the base URL from SNPedia
base_url = 'https://www.snpedia.com/index.php/'

In [ ]:
# Create URLs for each gene that I want to study
gene_urls = [base_url + rsid for rsid in bad_genes['rsid']]
for url in gene_urls:
    print(url, '\n')

In [ ]:
# Initialize Selenium
browser = webdriver.Chrome()

In [ ]:
import time

In [ ]:
# Write a function to visit the SNPedia URLs, click through to PubMed, 
# and retrieve the info on the articles for each gene

def scrape_abstracts(urls):
    
    #all_df = pd.DataFrame()
    rsid_list = []
    all_article_links = []
    all_article_title = []
    all_article_citation = []
    all_article_authors = []
    all_article_abstract = []
    
    for url in urls:
        browser.get(url) #load url
        rsid = browser.find_element_by_css_selector('.firstHeading').text
        links_elements = browser.find_elements_by_partial_link_text('PMID')
        link_urls = []
        
        for link in links_elements:
            link_urls.append(link.get_attribute('href')) # get the URLs to the PubMed pages
    
        for element in link_urls:
            browser.get(element) # follow each link element to PubMed
            time.sleep(.8)
            article_title = browser.find_element_by_xpath("//div[@class='cit']/../h1").text
            article_citation = browser.find_element_by_class_name('cit').text
            article_authors = browser.find_element_by_class_name('auths').text
            article_abstract = browser.find_element_by_class_name('abstr').text

            rsid_list.append(rsid)
            all_article_title.append(article_title)
            all_article_citation.append(article_citation)
            all_article_authors.append(article_authors)
            all_article_abstract.append(article_abstract)
            all_article_links.append(element)
            
        print(len(rsid_list) == len(link_urls) == len(all_article_title) == len(all_article_citation) == len(all_article_authors) == len(all_article_abstract))
     
    df = pd.DataFrame() # store the information
    df['rsid'] = rsid_list
    df['article_title'] = all_article_title
    df['article_citation'] = all_article_citation
    df['article_authors'] = all_article_authors
    df['article_abstract'] = all_article_abstract
    df['link'] = all_article_links
    df = df.drop_duplicates()

    df.index = range(len(df.index))
    
    return df

In [ ]:
abstracts_df = scrape_abstracts(gene_urls)

In [ ]:
abstracts_df

To save for later perusal, I'll export my web scrapings, complete with abstracts and hyperlinks, to a CSV file using the pandas DataFrame.to_csv method. 

In [ ]:
#DataFrame to CSV
export_csv = abstracts_df.to_csv(r'/Users/lorajohns/Documents/Python/DNA/DNA_articles.csv')

## Reading up on the medical literature

Now I have a handy CSV file, nicely formatted to read in Numbers, Excel, or PDF format, with citations to scientific articles analyzing and describing my genotypes with "significant" magnitudes and "bad" reputations. With the powerful tools Python provides, it's a great time to be alive for literal introspection. 